In [7]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf

tf.config.list_physical_devices()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [11]:
import numpy as np
from loader.CSVLoader import csv_loader

gt,wave = csv_loader()

name="pearson_30"

all_data = np.load(f"./experiment3/{name}.npy")
#test_data = np.load(f'./experiment3/{name}_test.npy')

# all_data_k = np.load(f"./experiment3/{name}_K.npy")
# test_data_k = np.load(f'./experiment3/{name}_K_test.npy')

# all_data_p = np.load(f"./experiment3/{name}_P.npy")
# test_data_p = np.load(f'./experiment3/{name}_P_test.npy')

# all_data_Mg = np.load(f"./experiment3/{name}_Mg.npy")
# test_data_Mg = np.load(f'./experiment3/{name}_Mg_test.npy')

# all_data_pH = np.load(f"./experiment3/{name}_pH.npy")
# test_data_pH = np.load(f'./experiment3/{name}_pH_test.npy')

all_data.shape

(1732, 10, 10, 30)

In [12]:
from creators.CNN3DCreator import  CNN3DNetworkParams
from creators.CRNNCreator import CRNNNetworkParams
from creators.ModelParams import Conv1DParams, Conv2DParams, Conv3DParams, DeepLayerParams, FlattenParams, GenericNeuralNetworkParams, RandomForestParams, RecurrentLayerParams, SVRParams
from creators.CNN1DCreator import  CNN1DNetworkParams
from creators.CNN2DCreator import  CNN2DNetworkParams
from prep.HelperBlocks import pipe, split_3d_to_timeseries
import tensorflow as tf
from prep.MSCBlock import msc_block, msc_block1d

from prep.Normlize import normalize_layers

drop=0.53
epochs=50
optimizer = tf.keras.optimizers.Adam
folds = 5

params_cnn2d = CNN2DNetworkParams(
    conv=[Conv2DParams(kernel=(5,5),pool_size=(2,2),filters=64,normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(3,3),pool_size=(2,2),filters=512, normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(2,2),pool_size=(2,2),filters=1024, normalization=True,activation="relu", dropout=drop),
          ],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: d]), 
        epochs=epochs,
        optimizer=optimizer
        )
)

params_crnn = CRNNNetworkParams(
    conv=[Conv3DParams(kernel=(1,10,10),pool_size=(1,10,10),filters=50,normalization=True,dropout=drop)],
    rec=[RecurrentLayerParams(type=tf.keras.layers.LSTM,units=125,bidirectional=True,dropout=drop),
         RecurrentLayerParams(type=tf.keras.layers.LSTM,units=125,bidirectional=True,dropout=drop)],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: split_3d_to_timeseries(d,10)
        ]),
        epochs=epochs,
        optimizer=optimizer
        )
)

params_cnn3d_low = CNN3DNetworkParams(
    conv=[Conv3DParams(kernel=(10,10,1),pool_size=(10,10,1),filters=128,normalization=True,activation="relu", dropout=drop),
          Conv3DParams(kernel=(1,1,6),pool_size=(1,1,6),filters=1024, normalization=True,activation="relu", dropout=drop),
          Conv3DParams(kernel=(1,1,5),pool_size=(1,1,5),filters=1024, normalization=True,activation="relu", dropout=drop),
          ],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: np.expand_dims(d, axis=-1),
        ]),
        epochs=epochs,
        optimizer=optimizer
        )
)

In [13]:
from ModelExperiment import ModelExperiment

experiment = ModelExperiment()
experiment.run_experiment(
    params_vector=[params_cnn2d,params_cnn2d,params_cnn2d,params_cnn2d],
    data=[all_data,all_data,all_data,all_data],
    gt=[gt["K"].values,gt["P"].values,gt["Mg"].values,gt["pH"].values],
    folds=folds,
    test_data=[all_data,all_data,all_data,all_data]
)

Epoch 1/50
44/44 [==============================] - 7s 154ms/step - loss: 227.6728 - mean_absolute_error: 227.6728 - score_metric: 16.3897 - lr: 0.0010 - val_loss: 228.3762 - val_mean_absolute_error: 228.3762 - val_score_metric: 19.9438 - val_lr: 0.0010
Epoch 2/50
44/44 [==============================] - 4s 92ms/step - loss: 226.2853 - mean_absolute_error: 226.2853 - score_metric: 16.2212 - lr: 0.0010 - val_loss: 228.7869 - val_mean_absolute_error: 228.7869 - val_score_metric: 20.1423 - val_lr: 0.0010
Epoch 3/50
44/44 [==============================] - 4s 97ms/step - loss: 224.0495 - mean_absolute_error: 224.0495 - score_metric: 16.3661 - lr: 0.0010 - val_loss: 223.8240 - val_mean_absolute_error: 223.8240 - val_score_metric: 19.2078 - val_lr: 0.0010
Epoch 4/50
44/44 [==============================] - 4s 98ms/step - loss: 220.1931 - mean_absolute_error: 220.1931 - score_metric: 15.5327 - lr: 0.0010 - val_loss: 218.7505 - val_mean_absolute_error: 218.7505 - val_score_metric: 18.2832 - va

In [14]:

experiment.models[0].save_history('./experiment3/cnn2d30_pearson_K.csv')
experiment.models[1].save_history('./experiment3/cnn2d30_pearson_P.csv')
experiment.models[2].save_history('./experiment3/cnn2d30_pearson_Mg.csv')
experiment.models[3].save_history('./experiment3/cnn2d30_pearson_pH.csv')

In [15]:
experiment.get_errors_report('./experiment3/errors_cnn2d_pearson30.csv')

In [ ]:
print(experiment.models[3].crossval_scores)
print(experiment.models[3].crossval_rs)
print(experiment.models[3].mae)

[0.89491722 0.83461045 0.86450696 0.77142927 0.93045412]
[0.10505724 0.16365829 0.13548495 0.2265189  0.06954481]
[0.18868703 0.186317   0.1858689  0.16857538 0.19020081]


In [ ]:
print(experiment.models[5].crossval_scores)
print(experiment.models[5].crossval_rs)
print(experiment.models[5].mae)

[0.92924521 0.93309339 0.90434612 0.91175416 0.97988553]
[0.07072827 0.06497107 0.09564543 0.08582077 0.02011335]
[0.19262939 0.19733602 0.19221746 0.18549364 0.19679249]
